In [19]:
import face_rec

In [20]:
import pandas as pd
import numpy as np
import cv2

### Step 1: Extract data from database

In [21]:
name = "test:register"
retrive_dict = face_rec.r.hgetall(name)

retrive_dict

{b'Barack Obama@Teacher': b'\xcd\xcc\xcc\xcc\x18[\xee?\x00\x00\x00\x80W\xde\xe1\xbf\x9a\x99\x99\x194J\xb8?\xcd\xcc\xcc\xcc\xd8x\xf4\xbf\x00\x00\x00\x00\xe0g\xd8?\x9a\x99\x99\x19\xa7O\xd2\xbf3333\xbe\x93\xc6?ffff\xaa\xc0\xbb\xbffff\xe6\xb3\xcf\x02\xc0\x00\x00\x00\x00"~\xf5\xbf\x00\x00\x00\x80\xdf\x19\xf2?ffff\xe2\x18\xab?3333~\xc5\xe4\xbf\x00\x00\x00\x00\xad\x92\xf1\xbf\x9a\x99\x99\x99i\xf3\xee?333\xb35]\x01\xc0\x00\x00\x00\x00\xd7(\xd7?\x9a\x99\x99\x99\x01\x02\xe2\xbf\xcd\xcc\xcc\xcc\xfbp\xdf\xbf333\xb3\xbe]\xe6\xbf\x9a\x99\x99\x19\x07\x1d\xf1\xbf\xcd\xcc\xcc\xcc\xf2\xaf\x04@\xcd\xcc\xcc\xccPJ\xef\xbf3333\xe5.\xdb?\xcd\xcc\xcc\xcc\x12F\xc1\xbf\x9a\x99\x99\x99~E\xd8\xbfffff\xfe\xa6\xe0\xbfffff\xa6\x07\xda\xbf\x9a\x99\x99\x99\xe8\xf1\xfa?3333eC\xb3?\x00\x00\x00\x00G\xc2\xe8?3333HT\xd4?3333\xdc\x13\xe4\xbf\x9a\x99\x99\x99\xcd!\xe1\xbfffff\x0f\x19\xf8\xbf\x9a\x99\x99\x19\xe0\x9a\xb5\xbf3333\\\xa5\xf1\xbf\x00\x00\x00\x00\x03\xa6\xf2?\x00\x00\x00\x00\xc8\x9f\xe8?\x9a\x99\x99\x99I\xa9\xd9?fff

In [22]:
retrive_series = pd.Series(retrive_dict)

# Convert binary data to numpy float32 arrays
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))

# Decode the index values
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))
retrive_series.index = index

# Convert series to dataframe and reset index
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role', 'facial_features']
retrive_df

,name_role,facial_features
0,Barack Obama@Teacher,"[-107374184.0, 1.862155, -0.0, -1.7645978, 1.5..."
1,Scarlett Johansson@Student,"[-107374184.0, 1.5662426, -1.5881868e-23, -1.7..."
2,Chris Evans@Student,"[-107374184.0, 1.8050444, 0.0, -1.9335388, 0.0..."
3,Morgan Freeman@Teacher,"[2.720083e+23, 1.7428625, -0.0, -1.7798909, 2...."
4,malani@Teacher,"[0.8200351, -1.0925149, -0.6477462, -0.8318283..."
5,Angelina Jolie@Student,"[0.0, 1.7161596, -107374184.0, 1.8843765, -0.0..."


In [23]:
retrive_df[['Name', 'Role']] = retrive_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)
retrive_df

,name_role,facial_features,Name,Role
0,Barack Obama@Teacher,"[-107374184.0, 1.862155, -0.0, -1.7645978, 1.5...",Barack Obama,Teacher
1,Scarlett Johansson@Student,"[-107374184.0, 1.5662426, -1.5881868e-23, -1.7...",Scarlett Johansson,Student
2,Chris Evans@Student,"[-107374184.0, 1.8050444, 0.0, -1.9335388, 0.0...",Chris Evans,Student
3,Morgan Freeman@Teacher,"[2.720083e+23, 1.7428625, -0.0, -1.7798909, 2....",Morgan Freeman,Teacher
4,malani@Teacher,"[0.8200351, -1.0925149, -0.6477462, -0.8318283...",malani,Teacher
5,Angelina Jolie@Student,"[0.0, 1.7161596, -107374184.0, 1.8843765, -0.0...",Angelina Jolie,Student


### Step 2: Get Realtime Predictions

In [24]:
cap = cv2.VideoCapture(0) # default 0, 1 for external camera

while True:
    ret, frame = cap.read()

    if ret == False:
        break

    pred_frame = face_rec.face_prediction(frame, retrive_df, 'facial_features', ['Name', 'Role'], thresh=0.5)

    cv2.imshow('frame', frame)
    cv2.imshow('prediction', pred_frame)

    if cv2.waitKey(1) == 27: # if I press esc button this condition will get triggered
        break

cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.